In [1]:
import pandas as pd

raw_data_location='s3://anomaly-detection-team-vypin/PS_20174392719_1491204439457_log.csv'

output_data_location_random_shuffled='./data.csv'
paysim_df=pd.read_csv(raw_data_location)
paysim_df.head()




,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [2]:
paysim_df.dtypes


step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [4]:
paysim_df_reduced=paysim_df.drop(columns=['nameDest','nameOrig','isFraud','isFlaggedFraud'])
paysim_df_reduced.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0
1,1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0
2,1,TRANSFER,181.00,181.0,0.00,0.0,0.0
3,1,CASH_OUT,181.00,181.0,0.00,21182.0,0.0
4,1,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0


In [5]:
paysim_df_hotencoded=pd.get_dummies(paysim_df_reduced)

In [6]:
paysim_df_hotencoded.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,170136.0,160296.36,0.0,0.0,0,0,0,1,0
1,1,1864.28,21249.0,19384.72,0.0,0.0,0,0,0,1,0
2,1,181.00,181.0,0.00,0.0,0.0,0,0,0,0,1
3,1,181.00,181.0,0.00,21182.0,0.0,0,1,0,0,0
4,1,11668.14,41554.0,29885.86,0.0,0.0,0,0,0,1,0


In [7]:
from sklearn.utils import shuffle
paysim_df_shuffled=shuffle(paysim_df_hotencoded)

In [8]:
paysim_df_shuffled.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
1668519,158,430188.35,0.0,0.0,27530719.77,27960908.12,0,1,0,0,0
5643328,396,2325.90,0.0,0.0,0.00,0.00,0,0,0,1,0
3560448,260,8427.95,0.0,0.0,0.00,0.00,0,0,0,1,0
1039921,94,27901.13,0.0,0.0,0.00,0.00,0,0,0,1,0
6133185,544,142218.74,127605.0,0.0,10029.10,152247.84,0,1,0,0,0


In [23]:

#paysim_df_shuffled.insert(0,'label',paysim_df_shuffled['isFraud'])



In [10]:
import boto3
import numpy as np

train_data, validation_data, test_data = np.split(paysim_df_shuffled.sample(frac=1, random_state=1729), [int(0.7 * len(paysim_df_shuffled)), int(0.9*len(paysim_df_shuffled))])  

train_data.to_csv('train.csv', index=False, header=False)
validation_data.to_csv('validation.csv', index=False, header=False)
test_data.to_csv('test.csv', index=False, header=False)



In [11]:
#Put the prepped file to S3
s3 = boto3.client('s3')
s3.upload_file('./train.csv', 'anomaly-detection-team-vypin', 'vishal/train.csv')
s3.upload_file('./validation.csv', 'anomaly-detection-team-vypin', 'vishal/train.csv')
s3.upload_file('./test.csv', 'anomaly-detection-team-vypin', 'vishal/train.csv')